## Facility Location Optimization for Dual-Capacity Drone Fleet in Last-Mile Delivery Operations


### Problem definition using Pyomo and solving it with Gurobi

# Combinatorial Optimization: Facility Location Optimization for Dual-Capacity Drone Fleet in Last-Mile Delivery Operations
## Problem Description:

The goal is to determine the optimal locations for facilities (e.g., drone hubs, recharging stations) to serve customer demand points effectively. Additionally, this problem involves choosing the appropriate type of drone for each delivery, balancing between **small-capacity drones** (for lighter, shorter deliveries) and **large-capacity drones** (for heavier loads or longer distances).

## Objective:

Minimize the total cost, which includes:

1. **Facility setup and operational costs.**
2. **Travel and operational costs** for each type of drone.
3. **Drone selection costs** based on the load and distance requirements for each customer.

## Constraints:

1. **Facility Assignment Constraint**: Each customer must be served by one facility and one type of drone.
2. **Facility Capacity Constraint**: Each facility has a total capacity limit for the number of drones it can support, with each type of drone occupying a different amount of space.
3. **Assignment Activation Constraint**: A customer can only be assigned to a facility that is open.
4. **Payload Capacity Constraint**:The assigned drone type must be able to carry the package weight.
5. **Distance/Range Constraint**: Small-capacity drones can only serve customers within range $R_{s}$, and large capacity drones within range $R_{l}$.
6. **Distinct Facility Assignment Constraint**: Each drone must be assigned to exactly one facility.


## Parameters:
$𝐹$ : Potential facility locations **(Set)** <br>
$𝐶$ : Customer locations **(Set)** <br>
$D_{𝑠}$ : Small capacity drones **(Set)** <br>
$𝐷_{𝑙}$ : Large capacity drones **(Set)** <br>
$d_{𝑖𝑗}$ : Distance between facility i and customer j, $𝑖\in𝐹$ & $𝑗\in𝐶$ **(2D Array)** <br>
$𝑓_{𝑖}$ : Fixed cost of operating facility i, $𝑖\in𝐹$ **(Array)** <br>
$𝑐_{𝑖𝑗}^s$ : Operational cost for serving customer j from facility i with an S-type drone **(2D Array)** <br>
$𝑐_{𝑖j}^l$ : Operational cost for serving customer j from facility i with an L-type drone **(2D Array)** <br>
$𝐾_{𝑖}$ : The capacity of each facility, the maximum number of drones it can support **(Array)** <br>
$𝑊_{𝑗}$ : Weight of customer’s j package **(Array)** <br>
$𝑤_{𝑠}$ : Weight factor for S-type drones for the facilities **(Constant)** <br>
$𝑤_{𝑙}$ : Weight factor for L-type drones for the facilities **(Constant)** <br>
$𝑃_{𝑠}$ : Payload capacity of S-type drones **(Constant)** <br>
$𝑃_{𝑙}$ : Payload capacity of L-type drones **(Constant)** <br>
$𝑅_{𝑠}$ : Max Range of S-type drones **(Constant)** <br>
$𝑅_{𝑙}$ : Max Range of L-type drones **(Constant)** <br>

## Decision Variables:
$x_{i} \in {0,1}$: Binary variable. Facility i is opened (1) or closed (0) <br>
$y_{ij}^s \in {0,1}$: Binary variable. If customer j is served by facility i using an S-type drone (1), otherwise (0) <br>
$y_{ij}^l \in {0,1}$: Binary variable. If customer j is served by facility i usingan L-type drone (1), otherwise (0) <br>


## ObjectiveFunction:
Minimize total cost. That includes the fixed costs and variable costs. <br>

$\displaystyle\sum_{i \in F} 𝑓_{𝑖} ∙ x_{i} + \displaystyle\sum_{i \in F} \displaystyle\sum_{j \in C} (𝑐_{𝑖𝑗}^s ∙ y_{ij}^s + (𝑐_{𝑖𝑗}^l + penalty \underline{ }large \underline{ }drone) ∙ y_{ij}^l)$ 

We give a penalty to the solver for choosing a large drone in order to discourage its selection unless absolutely necessary.

## Constraints:
### 1. Each customer must be served by one facility and one type of drone.
$\displaystyle\sum_{s \in D_{s}} \displaystyle\sum_{i \in F}  y_{ij}^s + \displaystyle\sum_{l \in D_{l}} \displaystyle\sum_{i \in F}  y_{ij}^l = 1, \quad \forall j \in C$

### 2. Each facility has  a total capacity limit for the number of drones it can support.
$\displaystyle\sum_{s \in Ds}\displaystyle\sum_{j \in C}(𝑤_{𝑠} ∙ y_{ij}^s) + \displaystyle\sum_{l \in Dl}\displaystyle\sum_{j \in C} (𝑤_{l} ∙ y_{ij}^l) \leq K_{i} ∙ x_{i}, \quad \forall i \in F$
 
### 3. A customer can only be assigned to a facility that is open.
$\quad$ For small drones s: <br>
$\quad \quad$ $y_{ij}^s \leq x_{i} \quad \forall s \in D_{s}, i \in F, j \in C$ <br>
$\quad$ For all large drones l: <br>
$\quad \quad$ $y_{ij}^l \leq x_{i}, \quad \forall l \in D_{l}, i \in F , j \in C$ <br>

### 4. The assigned drone type must be able to carry the package weight.
$\quad$ For small drones s: <br>
$\quad \quad$ $W_{j} ∙ y_{ij}^s \leq P_{s} \quad \forall s \in D_{s}, i \in F, j \in C$ <br>
$\quad$ For all large drones l: <br>
$\quad \quad$ $W_{j} ∙ y_{ij}^l \leq P_{l}, \quad \forall l \in D_{l}, i \in F , j \in C$ <br>

### 5. S-type drones can only serve customers within $R_{s}$ while L-type drones can serve customers within $R_{l}$
$\quad$ For small drones s: <br>
$\quad \quad$ $d_{ij} ∙ y_{ij}^s \leq R_{s} \quad \forall s \in D_{s}, i \in F, j \in C$ <br>
$\quad$ For all large drones l: <br>
$\quad \quad$ $d_{ij} ∙ y_{ij}^l \leq R_{l}, \quad \forall l \in D_{l}, i \in F , j \in C$ <br>


### 6. Each drone should be assigned to exactly one facility.
For Small Drones: <br>
$\quad$ $\displaystyle\sum_{i \in F}\displaystyle\sum_{j \in C}(y_{ij}^s) \leq 1, \quad \forall s \in D_{s}$
<br>
For Large Drones: <br>
$\quad$ $\displaystyle\sum_{i \in F}\displaystyle\sum_{j \in C}(y_{ij}^l) \leq 1, \quad \forall s \in D_{l}$





In [2]:
import pyomo.environ as pyomo

# Initialization of the model
model = pyomo.ConcreteModel()

# Define Sets
F = range(3)    # Facilities Set
C = range(4)    # Customers Set
Ds = range(5)   # S-type Drones Set
Dl = range(5)   # L-type Drones Set

# Parameters
# Distance between facility i and customer j 
d = [[30, 50, 40, 60],  
     [45, 60, 35, 70],
     [50, 40, 60, 80]]

# Fixed cost of operating facility i
f = [50, 60, 70]

# Operational cost for serving customer j from facility i with an S-type  drone
cs = [[10, 15, 20, 25],  
      [12, 14, 18, 22],
      [8, 10, 14, 20]]

# Operational cost for serving customer j from facility i with an L-type drone
cl = [[20, 25, 30, 35], 
      [25, 30, 35, 40],
      [18, 22, 28, 34]]

# The capacity of each facility, the maximum number of drones it can support
K = [10, 12, 15]

# Weight of customer’s j package
W = [4, 6, 8, 9]

# Weight factor for S-type drones for the facilities
ws = 1

# Weight factor for L-type drones for the facilities
wl = 2

# Payload capacity of S-type drones
Ps = 5

# Payload capacity of L-type drones
Pl = 10

# Max Range of S-type drones
Rs = 50

# Max Range of L-type drones
Rl = 100

# Decision Variables 
# Binary variable. Facility i is opened (1) or closed (0)
model.x = pyomo.Var(F, domain=pyomo.Binary)
# Binary variable. If customer j is served by facility i using an S-type drone (1), otherwise (0)
model.ys = pyomo.Var(Ds, F, C, domain=pyomo.Binary)
# Binary variable. If customer j is served by facility i using an L-type drone (1), otherwise (0)
model.yl = pyomo.Var(Dl, F, C, domain=pyomo.Binary)

# Define Objective Function 
def objective_function_rule(model):
    fixed_costs = sum(f[i] * model.x[i] for i in F)
    variable_costs = sum(cs[i][j] * model.ys[s, i, j] + cl[i][j] * model.yl[l, i, j] for s in Ds for l in Dl for i in F for j in C)
    return fixed_costs + variable_costs

model.objective = pyomo.Objective(rule=objective_function_rule, sense=pyomo.minimize)

# Constraints
# 1. Each customer must be served by one facility and one type of drone.
def is_customer_served(model, j):
    return sum(model.ys[s, i, j] for s in Ds for i in F) + sum(model.yl[l, i, j] for l in Dl for i in F) == 1

model.customer_served = pyomo.Constraint(C, rule=is_customer_served)

# 2. Each facility has a total capacity limit for the number of drones it can support.
def is_facility_maxed(model, i):
    return sum(ws * model.ys[s, i, j] for s in Ds for j in C) + sum(wl * model.yl[l, i, j] for l in Dl for j in C) <= K[i] * model.x[i]

model.facility_capacity = pyomo.Constraint(F, rule=is_facility_maxed)

# 3. A customer can only be assigned to a facility that is open. 
def is_small_facility_open(model, s, i, j):
    return model.ys[s, i, j] <= model.x[i]

model.small_facility_open = pyomo.Constraint(Ds, F, C, rule=is_small_facility_open)

def is_large_facility_open(model, l, i, j):
    return model.yl[l, i, j] <= model.x[i]

model.large_facility_open = pyomo.Constraint(Dl, F, C, rule=is_large_facility_open)

# 4. The assigned drone type must be able to carry the package weight.
def payload_capacity_small_rule(model, s, i, j):
    return W[j] * model.ys[s, i, j] <= Ps

model.payload_capacity_small = pyomo.Constraint(Ds, F, C, rule=payload_capacity_small_rule)

def payload_capacity_large_rule(model, l, i, j):
    return W[j] * model.yl[l, i, j] <= Pl

model.payload_capacity_large = pyomo.Constraint(Dl, F, C, rule=payload_capacity_large_rule)

# 5. S-type drones can only serve customers within Rs while L-type drones can serve customers within Rl
def range_limit_small_rule(model, s, i, j):
    return d[i][j] * model.ys[s, i, j] <= Rs

model.range_limit_small = pyomo.Constraint(Ds, F, C, rule=range_limit_small_rule)

def range_limit_large_rule(model, l, i, j):
    return d[i][j] * model.yl[l, i, j] <= Rl

model.range_limit_large = pyomo.Constraint(Dl, F, C, rule=range_limit_large_rule)


# # 6. Each small drone should be assigned to exactly one customer
# def distinct_drone_assignment_small_rule(model, s):
#     return sum(model.ys[s, i, j] for i in F for j in C) == 1

# model.distinct_drone_assignment_small = pyomo.Constraint(Ds, rule=distinct_drone_assignment_small_rule)

# # Each large drone should be assigned to exactly one customer
# def distinct_drone_assignment_large_rule(model, l):
#     return sum(model.yl[l, i, j] for i in F for j in C) == 1

# model.distinct_drone_assignment_large = pyomo.Constraint(Dl, rule=distinct_drone_assignment_large_rule)

# # 7. Each drone should be assigned to exactly one facility
# def distinct_facility_assignment_small_rule(model, s):
#     return sum(model.ys[s, i, j] for i in F for j in C) == 1

# model.distinct_facility_assignment_small = pyomo.Constraint(Ds, rule=distinct_facility_assignment_small_rule)

# def distinct_facility_assignment_large_rule(model, l):
#     return sum(model.yl[l, i, j] for i in F for j in C) == 1

# model.distinct_facility_assignment_large = pyomo.Constraint(Dl, rule=distinct_facility_assignment_large_rule)


# Solve the model
solver = pyomo.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

# Print Results
print("-----Facility Openings-----")
for i in F:
    print(f"Facility {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

print("-----Customer Assignments-----")
for j in C:
    assigned = False
    for i in F:
        for s in Ds:
            if model.ys[s, i, j]() > 0.5:
                print(f"Customer {j} is served by Facility {i} using Small Drone {s}.")
                assigned = True
        for l in Dl:
            if model.yl[l, i, j]() > 0.5:
                print(f"Customer {j} is served by Facility {i} using Large Drone {l}.")
                assigned = True
    if not assigned:
        print(f"Customer {j} is not assigned to any facility.")

print(f"Total Cost: {model.objective()}")


Set parameter Username
Set parameter LicenseID to value 2602449
Academic license - for non-commercial use only - expires 2025-12-20
Read LP format model from file C:\Users\ArisM\AppData\Local\Temp\tmpy_nvml65.pyomo.lp
Reading time = 0.01 seconds
x1: 367 rows, 123 columns, 723 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-4440 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 367 rows, 123 columns and 723 nonzeros
Model fingerprint: 0x9e8e8bb5
Variable types: 0 continuous, 123 integer (123 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 710.0000000
Presolve removed 357 rows and 115 columns
Presolve time: 0.01s
Presolved: 10 rows, 8 columns, 24 nonzeros
Found heur